In [17]:
import io
from collections import defaultdict
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import get_dataset_dir
k=4
n=5
user="7"

data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True,
               'min_support': n
               }
algo = KNNWithMeans(k=k, min_k=k, sim_options=sim_options, verbose=True)
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(filter(lambda x: x[0] == user, testset))

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [18]:
def get_top_n(predictions, n):
    # First map the predictions to user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def read_item_names():
    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = (line[1], line[2])
    return rid_to_name


In [19]:
# Read the mappings raw id <-> movie name
rid_to_name = read_item_names()
top_n = get_top_n(predictions, n)

# Print the recommended items
print("user: ", user)
for iid in top_n[user]:
    print('{:^5} {:<65} {:^5}'.format(iid[0], str(rid_to_name[str(iid[0])]), round(iid[1], 3)))

user:  7
 328  ('Conspiracy Theory (1997)', '08-Aug-1997')                         5  
1367  ('Faust (1994)', '01-Jan-1994')                                     5  
1512  ('World of Apu, The (Apur Sansar) (1959)', '05-Apr-1996')         4.968
1449  ('Pather Panchali (1955)', '22-Mar-1996')                         4.968
 315  ('Apt Pupil (1998)', '23-Oct-1998')                               4.949


In [16]:
res = []
for iid in top_n[user]:
    res.append('{} {} {}'.format(iid[0], str(rid_to_name[str(iid[0])]), round(iid[1], 3)))

result = {
    'user': user,
    'recommended items': res,
}
import json
json_file = open("output2.json",'w')
json_file.write(json.dumps(result))
json_file.close()